<a href="https://colab.research.google.com/github/mishragauravgm/fast-nlp/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install datasets transformers evaluate accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
import numpy as np
import datasets
import transformers, accelerate
import evaluate
from transformers import DataCollatorWithPadding

from sklearn.metrics import f1_score

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
CURR_DIR = '/content/drive/MyDrive/Colab Notebooks/NCRI Task' #current directory
SEED = 555
os.chdir(CURR_DIR)
print(os.getcwd())

/content/drive/MyDrive/Colab Notebooks/NCRI Task


### Loading the dataset created before

In [ ]:
dataset = datasets.load_from_disk("/content/drive/MyDrive/Colab Notebooks/NCRI Task/dataset.hf")

In [ ]:
model = 'distilbert-base-uncased-finetuned-sst-2-english' #Using this model as the base model

In [ ]:
#initializing the tokenizer and model
label2id = {'NEGATIVE':0,'POSITIVE':1}
id2label = {0:'NEGATIVE', 1:'POSITIVE'}
tokenizer = transformers.AutoTokenizer.from_pretrained(model)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model, num_labels=2, id2label=id2label, label2id=label2id)

def tokenize_the_review(datapoint):
    return tokenizer(datapoint["review"], truncation=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
#Tokenizing the existing dataset
tokenized_reviews = dataset.map(tokenize_the_review, batched=True)
tokenized_reviews = tokenized_reviews.rename_column("sentiment","label") #had issues with having the label column named as 'sentiment'

In [ ]:
#Creating a small dataset to check if my model architecture and training is working fine, before running it on all the data points
small_train = tokenized_reviews['train'].train_test_split(test_size = 0.01)
small_test = tokenized_reviews['test'].train_test_split(test_size=0.01)
tokenized_reviews_small = datasets.DatasetDict({
    'train': small_train['test'],
    'test': small_test['test']
})

In [ ]:
tokenized_reviews_small

DatasetDict({
    train: Dataset({
        features: ['review', 'label', 'input_ids', 'attention_mask'],
        num_rows: 400
    })
    test: Dataset({
        features: ['review', 'label', 'input_ids', 'attention_mask'],
        num_rows: 100
    })
})

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
#Making the metric function
f1 = evaluate.load("f1")
def compute_f1(eval_pred):
    predictions, labels = eval_pred
    prediction_label = np.argmax(predictions, axis=1)
    f1 = f1_score(prediction_label, labels)
    return {"f1":f1}

In [ ]:
#Freezing certain layers of the model to save time and to prevent catastrophic forgetting
for name, param in model.named_parameters():
    if name.startswith("distilbert"):
        param.requires_grad = False
        #Train only the last layers

training_args = transformers.TrainingArguments(
    output_dir="sentiment_detection",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True
)

trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_reviews["train"],
    eval_dataset=tokenized_reviews["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_f1
)

trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,0.256500,0.244401,0.903564
2,0.242200,0.240817,0.904292
3,0.237700,0.242225,0.906179
4,0.237600,0.239245,0.905946
5,0.245000,0.239058,0.907027


TrainOutput(global_step=12500, training_loss=0.24797456787109376, metrics={'train_runtime': 4231.1745, 'train_samples_per_second': 47.268, 'train_steps_per_second': 2.954, 'total_flos': 2.619733644268589e+16, 'train_loss': 0.24797456787109376, 'epoch': 5.0})